In [ ]:
from dotenv import load_dotenv
load_dotenv(override=True)

In [ ]:
# Initialize the LLM for use with router / structured output
from langchain.chat_models import init_chat_model
model_gemini_flash = init_chat_model("gemini-2.5-flash", model_provider="google_genai", timeout=30, temperature=0)
model_llama_groq = init_chat_model("llama-3.1-8b-instant", model_provider="groq", timeout=30, temperature=0)
model_gpt_4o_mini = init_chat_model("gpt-4o-mini", model_provider="openai", timeout=30, temperature=0)


In [ ]:
from typing_extensions import Literal
from langchain_core.messages import HumanMessage, SystemMessage
from langsmith import traceable
from pydantic import BaseModel, Field

class ClassificationResult(BaseModel):
  result:Literal["TOXIC","NOT_TOXIC"] = Field("llm's classification result if the user input text is 'TOXIC' or 'NON_TOXIC'")

@traceable
def toxicity_classifier(input:dict)->dict:
    instructions = (
      "Please review the user query below and determine if it contains any form of toxic behavior, "
      "such as insults, threats, or highly negative comments. Respond with 'TOXIC' if it does "
      "and 'NOT_TOXIC' if it doesn't."
    )
    classification_result = model_llama_groq.with_structured_output(ClassificationResult).invoke([SystemMessage(instructions), HumanMessage(input["text"])])

    return {"class":classification_result.result}

    

In [ ]:
toxicity_classifier({"text": "This is unacceptable. I want to speak to the manager."})

In [ ]:
from langsmith import Client
ls_client = Client()

examples = [
  {
    "inputs": {"text": "Shut up, idiot"},
    "outputs": {"label": "TOXIC"},
  },
  {
    "inputs": {"text": "You're a wonderful person"},
    "outputs": {"label": "NOT_TOXIC"},
  },
  {
    "inputs": {"text": "This is the worst thing ever"},
    "outputs": {"label": "TOXIC"},
  },
  {
    "inputs": {"text": "I had a great day today"},
    "outputs": {"label": "NOT_TOXIC"},
  },
  {
    "inputs": {"text": "Nobody likes you"},
    "outputs": {"label": "TOXIC"},
  },
  {
    "inputs": {"text": "This is unacceptable. I want to speak to the manager."},
    "outputs": {"label": "NOT_TOXIC"},
  },
]

dataset_name="Toxic Queries"
if not ls_client.has_dataset(dataset_name=dataset_name):
    dataset = ls_client.create_dataset(dataset_name=dataset_name)
    ls_client.create_examples(
        dataset_id=dataset.id,
        examples=examples,
        )

In [ ]:
def correct(inputs:dict, outputs:dict, reference_outputs:dict)->bool:
    return outputs["class"] == reference_outputs["label"]

In [ ]:
results = ls_client.evaluate(
    toxicity_classifier,
    data=dataset.name,
    evaluators=[correct],
    experiment_prefix="llama-3.1-8b-instant, groq",  # optional, experiment name prefix
    description="Testing the baseline system.",  # optional, experiment description
    max_concurrency=4,  # optional, add concurrency
)